In [ ]:
import warnings
import math

import numpy as np
import pandas as pd
import datetime as dt
from datetime import date

import plotly.express as px

from tqdm import tqdm

warnings.filterwarnings('ignore')

# Ботяра

бла бла бла

## Заполнение датафрэйма

Взял Сбер

In [ ]:
df = pd.read_csv('sr_full_date.csv', sep=",")
df = df.sort_values(by=['ticker', 'tradedate', 'tradetime']).drop_duplicates().reset_index().drop('index', axis=1)
df['datetime'] = pd.to_datetime(df['tradedate'] + ' ' + df['tradetime']) 
df = df.drop(['tradedate', 'tradetime'], axis=1) 
df = df[df['clgroup'] == 'YUR'].reset_index().drop('index', axis=1)
df = df[['datetime', 'pos']]
data_pos = df[df['datetime'].dt.date > pd.to_datetime('2020-06-21').date()].reset_index().drop('index', axis=1)
#data_pos

In [ ]:
df = pd.read_csv('SBER_full_date_price.csv', sep=",")
df.reset_index(inplace=True)

df.rename(columns={'TRADEDATE': 'datetime'}, inplace=True)
df['datetime'] = pd.to_datetime(df['datetime'], dayfirst=True)
df = df[df['BOARDID']=='TQBR']
#for column in df.columns:
#    print(df[f'{column}'])
df = df[['datetime', 'WAPRICE']]
df.rename(columns={'WAPRICE': 'price'}, inplace=True)
df['price'] *= 10 # 10лотов
data_price = df
#data_price

In [ ]:
price_dict = dict(zip(data_price['datetime'].dt.date, data_price['price']))

data_pos['price'] = data_pos['datetime'].dt.date.map(price_dict)

all_data = data_pos.sort_values('datetime').drop_duplicates().reset_index().drop('index', axis=1)
#all_data

In [ ]:
all_data_copy = all_data.copy()

all_data_copy['date'] = all_data_copy['datetime'].dt.date
all_data_copy = all_data_copy[all_data_copy['date'] < pd.to_datetime('2021-11-01').date()]
df = all_data_copy.copy()
#print(df[df['datetime']==pd.to_datetime('2020-10-21 18:44:59')])

In [ ]:
window_size = 1000  # Пример размера окна
df['pos_ma'] = df['pos'].rolling(window=window_size).mean()
mean_pos_ma = df['pos_ma'].mean()
std_pos_ma = df['pos_ma'].std()
pos_ma_upper = mean_pos_ma + 2 * std_pos_ma
pos_ma_lower = mean_pos_ma - 2.2 * std_pos_ma
print(mean_pos_ma, std_pos_ma)

In [ ]:
fig = px.line(df, x='datetime', y='pos_ma', title=f'График pos MA{window_size} по времени', labels={'datetime': 'Дата и время', 'pos_ma': f'Pos MA{window_size}'})
fig.add_shape(
    dict(
        type="line",
        x0=df['datetime'].min(),
        x1=df['datetime'].max(),
        y0=pos_ma_upper,
        y1=pos_ma_upper,
        line=dict(color="green")
    )
)
fig.add_shape(
    dict(
        type="line",
        x0=df['datetime'].min(),
        x1=df['datetime'].max(),
        y0=pos_ma_lower,
        y1=pos_ma_lower,
        line=dict(color="red")
    )
)
fig.update_xaxes(type='category')
fig.show()

In [ ]:
outliers_up = pd.DataFrame(columns=['datetime', 'pos', 'price', 'date'])
outliers_down = pd.DataFrame(columns=['datetime', 'pos', 'price', 'date'])
for i in range(len(df['pos'])-1):
    if (df['pos_ma'][i + 1] > (pos_ma_upper)) and (df['pos_ma'][i] < (pos_ma_upper)):
        outliers_up.loc[len(outliers_up)] = df.iloc[i + 1]
    elif (df['pos_ma'][i + 1] < (pos_ma_upper)) and (df['pos_ma'][i] > (pos_ma_upper)):
        outliers_down.loc[len(outliers_down)] = df.iloc[i + 1]
    elif (df['pos_ma'][i + 1] < (pos_ma_lower)) and (df['pos_ma'][i] > (pos_ma_lower)):
        outliers_down.loc[len(outliers_down)] = df.iloc[i + 1]
    elif (df['pos_ma'][i + 1] > (pos_ma_lower)) and (df['pos_ma'][i] < (pos_ma_lower)):
        outliers_up.loc[len(outliers_up)] = df.iloc[i + 1]
fig = px.line(df, x='datetime', y='price', title='График цен по времени', labels={'datetime': 'Дата и время', 'price': 'Цена'})
fig.add_trace(px.scatter(outliers_up, x='datetime', y='price').update_traces(marker=dict(color='green')).data[0])
fig.add_trace(px.scatter(outliers_down, x='datetime', y='price').update_traces(marker=dict(color='red')).data[0])
fig.update_xaxes(type='category')
fig.show()

## Класс стратегии

In [ ]:
class MyPosition:

    def __init__(self, amount: float, price_current: float, short: bool):
        self._amount: float = amount
        self._price_current: float = price_current
        self._acc_fees: float = 0
        self._short: bool = short

    def update_state(self, price: float) -> None:
        self._price_current = price
        if self._short:
            self._acc_fees += abs(self._amount) * self._price_current * (0.00065) #комса за перенос короткой позиции

    def balance(self) -> float:
        return self._amount * self._price_current

    def __str__(self) -> str:
        return f"amount: {self._amount}, price_current: {self._price_current}"

    def __repr__(self) -> str:
        return self.__str__()

In [ ]:
class Strategy:

    params: dict = {
        'FEE': 0.0004,
    }
     
    def __init__(self,
                 states: pd.DataFrame,
                 start_equity: float
                ):
        self._position: MyPosition = None
        self._states: pd.DataFrame = states
        self._data = []
        self._equity: float = start_equity
        self._margin_equity: float = 0

    def calc_timestamps(self, STD_COUNT_UP: int, STD_COUNT_DOWN: int, MA_COUNT: int) -> (pd.DatetimeIndex, pd.DatetimeIndex):

        window_size = MA_COUNT
        states: pd.Dataframe = self._states.copy()
        states['pos_ma'] = states['pos'].rolling(window=window_size).mean()

        mean_pos_ma = states['pos_ma'].mean()
        std_pos_ma = states['pos_ma'].std()

        pos_ma_upper = mean_pos_ma + STD_COUNT_UP * std_pos_ma
        pos_ma_lower = mean_pos_ma - STD_COUNT_DOWN * std_pos_ma

        timestamps_up = []
        timestamps_down = []

        for i in tqdm(range(len(states['pos_ma'])-1), disable = True):
            state = states.iloc[i]
            next_pos_ma = states['pos_ma'].iloc[i + 1]

            if (next_pos_ma > pos_ma_upper) and (state['pos_ma'] < pos_ma_upper):
                timestamps_up.append(pd.to_datetime(state['datetime']))
            elif (next_pos_ma < pos_ma_upper) and (state['pos_ma'] > pos_ma_upper):
                timestamps_down.append(pd.to_datetime(state['datetime']))
            elif (next_pos_ma < pos_ma_lower) and (state['pos_ma'] > pos_ma_lower):
                timestamps_down.append(pd.to_datetime(state['datetime']))
            elif (next_pos_ma > pos_ma_lower) and (state['pos_ma'] < pos_ma_lower):
                timestamps_up.append(pd.to_datetime(state['datetime']))
        return timestamps_up, timestamps_down

    def run(self, RISK: float, STD_COUNT_UP: int, STD_COUNT_DOWN: int, MA_COUNT: int):
        
        timestamps_up = []
        timestamps_down = []

        (timestamps_up, timestamps_down) = self.calc_timestamps(STD_COUNT_UP, STD_COUNT_DOWN, MA_COUNT)

        timestamps_up = pd.to_datetime(timestamps_up)
        timestamps_down = pd.to_datetime(timestamps_down)

        long_position_opened = False
        short_position_opened = False

        #for i in tqdm(range(len(self._states))):
        #    state = self._states.loc[i]
        for i, state in tqdm(self._states.iterrows(), disable = True):
            if self._position:
                if (i != 0) and (state['price'] != self._states['price'][i-1]):
                    self._position.update_state(state['price'])

                self._data.append([
                state['datetime'],
                state['price'],
                self._equity - self._position._acc_fees
                ])
                
                if (state['datetime'] in timestamps_up) and short_position_opened:
                    #print('shortc', state['datetime'])
                    self.close_short()
                    short_position_opened = False
                if (state['datetime'] in timestamps_down) and long_position_opened:
                    #print('longc', state['datetime'])
                    self.close_long()
                    long_position_opened = False
            else:
                if (state['datetime'] in timestamps_up) and (long_position_opened == False):
                    #print('longo', state['datetime'])
                    self.open_long(state, RISK)
                    long_position_opened = True
                if (state['datetime'] in timestamps_down) and (short_position_opened == False):
                    #print('shorto', state['datetime'])
                    self.open_short(state, RISK)
                    short_position_opened = True

                self._data.append([
                state['datetime'],
                state['price'],
                self._equity
                ])

        return pd.DataFrame(self._data, columns=['datetime', 'price', 'equity'])
        
    def open_short(self, state, RISK: float):
        if self._position:
            raise Exception(f'Cannot open position, already have one {self._position}')

        if self._equity <= 0:
            raise Exception(f'Cannot open short position {self._equity}')
        
        self._margin_equity += (math.floor((1 - self.params['FEE']) * RISK * self._equity / state['price']) * state['price'] - self.params['FEE'] * self._equity)

        amount = (-1) * math.floor((1 - self.params['FEE']) * RISK * self._equity / state['price'])

        self._position = MyPosition(amount, state['price'], True)
    
    def open_long(self, state, RISK: float):
        if self._position:
            raise Exception(f'Cannot open position, already have one {self._position}')

        if self._equity <= 0:
            raise Exception(f'Cannot open long position  balance = {self._equity}')

        self._margin_equity -= (math.floor((1 - self.params['FEE']) * RISK * self._equity / state['price']) * state['price'] + self.params['FEE'] * self._equity)

        amount = math.floor((1 - self.params['FEE']) * RISK * self._equity / state['price'])

        self._position = MyPosition(amount, state['price'], False)

    def close_short(self):
        if self._position._short == 0:
            raise Exception(f'Cannot close short position, it is long')

        #print(self._margin_equity, (1 - self.params['FEE']) * self._position.balance(),  self._position._acc_fees)
        self._equity += (self._margin_equity + (1 - self.params['FEE']) * self._position.balance() - self._position._acc_fees)
        self._margin_equity = 0
        self._position = None

    def close_long(self):
        if self._position._short == 1:
            raise Exception(f'Cannot close long position, it is short')

        self._equity += (self._margin_equity + (1 - self.params['FEE']) * self._position.balance() - self._position._acc_fees)
        self._margin_equity = 0
        self._position = None

In [ ]:
strategy = Strategy(all_data_copy, 100000)
df = pd.DataFrame(strategy.run(1, 1.5, 2, 1000))
px.line(df, x='datetime', y='equity').update_xaxes(type='category').show()

In [ ]:
std_counts_up = np.arange(0.5, 2.5, 0.5)
std_counts_down = np.arange(0.5, 2.5, 0.5)
ma_counts = np.arange(100, 1100, 200)


parameters = []
final_total_balances = []

for std_count_up in std_counts_up:
    print(std_count_up)
    for std_count_down in std_counts_down:
        for ma_count in ma_counts:
            strategy = Strategy(all_data_copy, 10000000)
            df = pd.DataFrame(strategy.run(1, std_count_up, std_count_down, ma_count))
            df['equity'] = df['equity'] / df['equity'].iloc[0]
            #print(std_count_up, std_count_down, ma_count, df['equity'])   
            parameters.append((std_count_up, std_count_down, ma_count))
            final_total_balance = df['equity'].iloc[-1]
            final_total_balances.append(final_total_balance)

# Find the maximum total balance and its corresponding parameters
max_total_balance = max(final_total_balances)
max_total_balance_index = final_total_balances.index(max_total_balance)
corresponding_parameters = parameters[max_total_balance_index]

# Print the maximum total balance and its corresponding parameters
print(f"Maximum Total Balance: {max_total_balance}")
print(f"Corresponding Parameters (std_count, std_timerange): {corresponding_parameters}")

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

std_counts_up = np.arange(0.5, 1.5, 0.5)
std_counts_down = np.arange(0.5, 1.5, 0.5)
ma_counts = np.arange(1000, 1100, 200)

parameters = []
final_total_balances = []

for std_count_up in std_counts_up:
    for std_count_down in std_counts_down:
        for ma_count in ma_counts:
            strategy = Strategy(all_data_copy, 10000000)
            df = pd.DataFrame(strategy.run(1, std_count_up, std_count_down, ma_count))
            df['equity'] = df['equity'] / df['equity'].iloc[0]
            parameters.append((std_count_up, std_count_down, ma_count))
            final_total_balance = df['equity'].iloc[-1]
            final_total_balances.append(final_total_balance)

# Преобразование в DataFrame для удобства работы с данными
df_plotly = pd.DataFrame(parameters, columns=['std_count_up', 'std_count_down', 'ma_count'])
df_plotly['final_total_balance'] = final_total_balances

# Создание трехмерного графика в Plotly
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])

scatter = go.Scatter3d(
    x=df_plotly['std_count_up'],
    y=df_plotly['std_count_down'],
    z=df_plotly['ma_count'],
    mode='markers',
    marker=dict(
        size=8,
        color=df_plotly['final_total_balance'],
        colorscale='Viridis',
        colorbar=dict(title='Final Total Balance')
    )
)

fig.add_trace(scatter)

# Наименование осей
fig.update_layout(scene=dict(
                    xaxis_title='std_count_up',
                    yaxis_title='std_count_down',
                    zaxis_title='ma_count')
                 )

# Отображение графика
fig.show()